In [48]:
!nvidia-smi

Thu Oct 22 06:11:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.31       Driver Version: 440.31       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:06:00.0 Off |                    0 |
| N/A   34C    P0    49W / 250W |  21755MiB / 22919MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:2F:00.0 Off |                    0 |
| N/A   33C    P0    49W / 250W |  21766MiB / 22919MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [15]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()
c = []
# for d in ['/device:GPU:2', '/device:GPU:3']:

with tf.device('/device:GPU:2'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
    c.append(tf.matmul(a, b))
    
with tf.device('/device:GPU:3'):
    a = tf.constant([1.0, 4.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
    c.append(tf.matmul(a, b))
    

with tf.device('/device:GPU:2'):
    sum = tf.add_n(c)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(sum))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:2 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:3 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P40, pci bus id: 0000:06:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: Tesla P40, pci bus id: 0000:2f:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:2 -> device: 2, name: Tesla P40, pci bus id: 0000:86:00.0, compute capability: 6.1
/job:localhost/replica:0/task:0/device:GPU:3 -> device: 3, name: Tesla P40, pci bus id: 0000:d8:00.0, compute capability: 6.1

[[ 50.  64.]
 [ 98. 128.]]


In [1]:
from utils.graph import GraphConstruct
from utils.metrics import compute_acc
from utils.data_prepare import get_data
from utils.batch_generate import Batch_iter

from utils.multi_gpu import average_gradients,assign_to_device

import numpy as np

import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'

num_nodes=370
edge=np.load("edge.npy")

Graph=GraphConstruct(num_nodes,edge,max_hop=40,strategy='distance')
A=Graph.A

print(A.shape)



Instructions for updating:
non-resource variables are not supported in the long term
(40, 370, 370)


In [2]:
######################################搭建网络######################################
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import os 
os.environ["CUDA_VISIBLE_DEVICES"]='2' 

from tensorflow.keras import layers

import numpy as np


def conv2d(x,num_filter,kernel_size=(1,1),padding='same',activation=tf.nn.relu,use_bn=False):
    if use_bn:
        x=layers.BatchNormalization(axis=1)(x)
    x=tf.layers.conv2d( x,
                    filters=num_filter,   
                    kernel_size=kernel_size,        
                    strides=(1, 1),
                    padding=padding,
                    activation=activation,
                          )
    return x

def point_importance(x):
    
    Im=tf.get_variable('Im',x.shape,initializer=tf.ones_initializer())
    x=Im*x
    
    return x

def gcn_block(x,A,name="O_vision"):
    """
    Args:
        x : (n,t,h,w,c)
        A : (k,h,w)        
    return
        x : (n,t,h,w,c)
    """   
    n,t,h,w,c=x.shape

    k,h,w = A.shape

    # forward feature when x, backwards when x=transpose(x)
    with tf.variable_scope("gcn_block_%s"%name):
        
        A=point_importance(A)
        
        x=tf.einsum('nthwc, khw -> nthwc',x,A)
        b=tf.get_variable('b',[t,h,w,c],initializer=tf.zeros_initializer())
        x=x+b
        
        x_o = conv2d(x,1,(1,w),'valid')
        x_o = tf.tile(x_o,[1,1,1,w,1])
        w_o=tf.get_variable('wo',[t,h,w,c],initializer=tf.ones_initializer())
        x_o = tf.einsum('nthwc, thwc -> nthwc',x_o,w_o)
        b_o=tf.get_variable('bo',[t,h,w,c],initializer=tf.zeros_initializer())
        x_o = x_o+b_o

        
        x_d  = conv2d(x,1,(h,1),'valid')
        x_d = tf.tile(x_d,[1,1,h,1,1])
        w_d=tf.get_variable('wd',[t,h,w,c],initializer=tf.ones_initializer())
        x_d = tf.einsum('nthwc, thwc -> nthwc',x_d,w_d)
        b_d=tf.get_variable('bd',[t,h,w,c],initializer=tf.zeros_initializer())
        x_d = x_d+b_d
        
        x=tf.add(x_o,x_d)
        x=tf.nn.relu(x)
        

    return x


def static_var(varname, value):
    def decorate(func):
        setattr(func, varname, value)
        return func
    return decorate

# @static_var("gcc_layers", [0])
def tcn_block(x,num_filter=1,kernel_size=(19,1,1),padding='same',blockId='0'):
    """
    Args:
        x : (n,t,h,w,c)       
    return
        x : (n,t,h,w, num_filter)
    """
    n,t,h,w,c=x.shape
        
#     i = gcc.gcc_layers[-1]+1
#     gcc.gcc_layers.append(i)
    
    with tf.variable_scope("tcn_block%s"%blockId):
#         x=layers.ConvLSTM2D(num_filter,kernel_size=kernel_size, padding='same',data_format='channels_last',)(x)
#         x=layers.Conv3D(num_filter,kernel_size=kernel_size,padding=padding,data_format='channels_last')(x)
        w_3d = tf.get_variable('w3d',[t,c,num_filter],initializer=tf.ones_initializer())
    
        if padding=='same':
            x =  tf.einsum('nthwc, tcf -> nthwf',x,w_3d)
            n,t,h,w,f=x.shape
            b_3d = tf.get_variable('b3d',[t,h,w,f],initializer=tf.zeros_initializer())
            x=x+b_3d
        elif padding=='valid':
            x =  tf.einsum('nthwc, tcf -> nhwf',x,w_3d)
            n,h,w,f=x.shape
            b_3d = tf.get_variable('b3d',[h,w,f],initializer=tf.zeros_initializer())
            x=x+b_3d
            x=tf.reshape(x,[-1,1,h,w,f])
        x=tf.nn.relu(x)
        
    return x


def embed_his_chara_block(x,cha):
    """
    Args:
        x : (n,t,h,w,c)  
        cha : (n,t,hc)
    return
        x : (n,t,h,w,c+hc)
    """
    n,t,h,w,c = x.shape

    nc,tc,hc = cha.shape
    
    with tf.variable_scope("embed_his_chara_block"):
        cha = tf.reshape(cha,[-1,tc,1,1,hc])
        cha = tf.tile(cha,(1,1,h,w,1))
        x=tf.concat([x,cha],4)

    return x

def embed_fur_chara_block(x,cha):
    """
    Args:
        x : (n,t,h,w,1)  
        cha : (n,hc)
    return
        x : (n,t+hc,h,w,1)
    """
    n,t,h,w,c = x.shape

    nc,hc = cha.shape
    
    with tf.variable_scope("embed_fur_chara_block"):
        cha = tf.reshape(cha,[-1,hc,1,1,1])
        cha = tf.tile(cha,(1,1,h,w,1))
        x=tf.concat([x,cha],1)

    return x


def CSTGCN(x_batch,A_,x_cha_batch,y_cha_batch,reuse):
    #########################################构造网络#################################
    
    with tf.variable_scope("CSTGCN",reuse=reuse):
        #gcn
        output=gcn_block(x_batch,A_) #(n,t,h,w,c)  

        x_T=tf.transpose(x_batch,(0,1,3,2,4))  #(n,t,h,w,c)  
        output_T=gcn_block(x_T,A_,"D_vision") #(n,t,h,w,c)  

        output=tf.add(output,output_T) #(n,t,h,w,c)  
        output=tf.nn.relu(output) #(n,t,h,w,c)  

        #embed_his_chara
        output=embed_his_chara_block(output,x_cha_batch) #(n,t,h,w,c+hc)  

        #tcn_embed_fur_chara
        output=tcn_block(output,3,kernel_size=(timestep,1,1),blockId=0) #(n,t,h,w,3)  
        output=tcn_block(output,1,kernel_size=(timestep,1,1),blockId=1) #(n,t,h,w,1)  ###这里reuse没生效！！！！！！！！！！！！

        output=embed_fur_chara_block(output,y_cha_batch)  #(n,t+hc,h,w,1)  


        #tcn_inference
        t_new=output.get_shape()[1].value
        output=tcn_block(output,1,kernel_size=(t_new,1,1),padding='valid',blockId=2) #(n,1,h,w,1)  
        output=tf.squeeze(output,1) #(n,h,w,1) 
    
    return output

epochs=100
batch_size=6*4
num_gpus=4
per_gpu_batch_size=int(batch_size/num_gpus)

timestep=19
c=1
hc=1



train_x,train_y,train_x_cha,train_y_cha, test_x,test_y,test_x_cha,test_y_cha = get_data(timestep)
batch_iter=Batch_iter(train_x,train_y,train_x_cha,train_y_cha,batch_size,epochs)

x (228, 19, 370, 370, 1) 
y (228, 370, 370, 1) 
x_cha (228, 19, 1) 
y_cha (228, 1)


In [6]:
tf.reset_default_graph()

with tf.device('/cpu:0'):
    
    tower_grads = []
    reuse_vars = False
    learning_rate = 0.01
    
    x_batch=tf.placeholder(tf.float32,[None,timestep,370,370,c])
    A_=tf.placeholder(tf.float32,[40,370,370])
    y_batch=tf.placeholder(tf.float32,[None,370,370,c])
    x_cha_batch=tf.placeholder(tf.float32,[None,timestep,hc])
    y_cha_batch=tf.placeholder(tf.float32,[None,hc])
    
    # Loop over all GPUs and construct their own computation graph
    for i in range(4):
        with tf.device(assign_to_device('/gpu:{}'.format(i), ps_device='/cpu:0')):

            # Split data between GPUs
            _x = x_batch[i * per_gpu_batch_size: (i+1) * per_gpu_batch_size]
            _y = y_batch[i * per_gpu_batch_size: (i+1) * per_gpu_batch_size]
            _x_cha = x_cha_batch[i * per_gpu_batch_size: (i+1) * per_gpu_batch_size]
            _y_cha = y_cha_batch[i * per_gpu_batch_size: (i+1) * per_gpu_batch_size]

            # Because Dropout have different behavior at training and prediction time, we
            # need to create 2 distinct computation graphs that share the same weights.
            

            # Create a graph for training
            output = CSTGCN(_x,A_,_x_cha,_y_cha
                                    ,reuse=reuse_vars
                                  #, is_training=True
                                     )
            # Create another graph for testing that reuse the same weights
            output_test = CSTGCN(_x,A_,_x_cha,_y_cha
                                   ,reuse=True
                                   #, is_training=False
                                  )

            # Define loss and optimizer (with train logits, for dropout to take effect)
            trainloss = tf.reduce_sum(tf.square(_y - output))
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            grads = optimizer.compute_gradients(trainloss)
            
            # Only first GPU compute accuracy
            if i == 0:
                # Evaluate model (with test logits, for dropout to be disabled)
                loss = tf.reduce_sum(tf.square(_y - output_test))
                accurancy,accurancy1 = compute_acc(_y,output_test)

            reuse_vars = True
            tower_grads.append(grads)

    tower_grads = average_gradients(tower_grads)
    train = optimizer.apply_gradients(tower_grads)

In [7]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P40, pci bus id: 0000:86:00.0, compute capability: 6.1



In [ ]:
###训练
loss_list=[[],[]]
acc_list=[[],[]]
acc1_list=[[],[]]
for i,(train_x_batch,train_y_batch,train_x_cha_batch,train_y_cha_batch) in enumerate(batch_iter):    

#     if i%100==0:
#         [ls,acc] = sess.run([loss,accurancy],feed_dict={x:x_batch,y:y_batch})

    
    
#     sess.run(train,feed_dict={img_batch:x,lab_batch:l,out_batch:o})
    _,train_loss,train_acc,train_acc1,train_output = sess.run([train,loss,accurancy,accurancy1,output_test],\
                                                    feed_dict={x_batch:train_x_batch,y_batch:train_y_batch,\
                                                               x_cha_batch:train_x_cha_batch,y_cha_batch:train_y_cha_batch,\
                                                               A_:A})
    
    print(train_y_batch[0,0,:10,0].astype(int).tolist())
    print(train_output[0,0,:10,0].astype(int).tolist())
    
    test_loss,test_acc,test_acc1= sess.run([loss,accurancy,accurancy1],\
                                                  feed_dict={x_batch:test_x,y_batch:test_y,\
                                                             x_cha_batch:test_x_cha,y_cha_batch:test_y_cha, \
                                                             A_:A})
    
    print('###### %s batch Train loss:%s acc:%s acc1:%s  Test loss:%s acc:%s acc1:%s\n'%\
                  (int(i),train_loss,train_acc,train_acc1,test_loss,test_acc,test_acc1))
    
    loss_list[0].append(train_loss); loss_list[1].append(test_loss)
    acc_list[0].append(train_acc); acc_list[1].append(test_acc)
    acc1_list[0].append(train_acc1); acc1_list[1].append(test_acc1)
#     if 1>train_acc>=0.98:
#         break
#     if i==3:
#         break

[8, 8, 30, 8, 19, 49, 40, 30, 8, 13]
[123, 121, 122, 123, 123, 124, 122, 121, 121, 122]
###### 0 batch Train loss:28650052000.0 acc:-96.413574 acc1:-96.413574  Test loss:11663801000.0 acc:-54.373363 acc1:-54.373363

[3, 0, 6, 9, 8, 7, 6, 19, 14, 10]
[218, 218, 218, 218, 218, 218, 218, 218, 218, 218]
###### 1 batch Train loss:23109773000.0 acc:-79.22423 acc1:-79.22423  Test loss:10803232000.0 acc:-52.30787 acc1:-52.30787

[7, 20, 11, 13, 14, 82, 12, 10, 11, 11]
[197, 197, 197, 197, 197, 197, 197, 197, 197, 197]
###### 2 batch Train loss:22706150000.0 acc:-76.85618 acc1:-76.85618  Test loss:10035864000.0 acc:-50.371647 acc1:-50.371647

[15, 6, 11, 7, 20, 30, 17, 6, 10, 13]
[204, 204, 204, 204, 204, 204, 204, 204, 204, 204]
###### 3 batch Train loss:16666384000.0 acc:-72.50813 acc1:-72.50813  Test loss:9309217000.0 acc:-48.465546 acc1:-48.465546

[8, 7, 13, 17, 27, 25, 23, 9, 5, 14]
[189, 189, 189, 189, 189, 189, 189, 189, 189, 189]
###### 4 batch Train loss:24541493000.0 acc:-81.08851 ac

[14, 6, 14, 4, 13, 32, 20, 13, 20, 9]
[39, 39, 40, 39, 40, 42, 40, 40, 40, 40]
###### 40 batch Train loss:1773962500.0 acc:-21.56745 acc1:-21.56745  Test loss:672785500.0 acc:-11.462846 acc1:-11.462846

[10, 7, 18, 10, 17, 15, 14, 2, 9, 2]
[61, 61, 61, 61, 61, 64, 62, 61, 61, 61]
###### 41 batch Train loss:1159545900.0 acc:-14.827704 acc1:-14.827704  Test loss:639141100.0 acc:-11.097649 acc1:-11.097649

[8, 10, 13, 17, 12, 44, 33, 17, 13, 11]
[52, 52, 52, 52, 52, 55, 53, 52, 52, 52]
###### 42 batch Train loss:1737646100.0 acc:-19.349874 acc1:-19.349874  Test loss:607687800.0 acc:-10.745908 acc1:-10.745908

[10, 15, 42, 37, 40, 45, 39, 16, 25, 9]
[80, 80, 80, 80, 81, 83, 81, 81, 81, 81]
###### 43 batch Train loss:1045773400.0 acc:-13.849594 acc1:-13.849594  Test loss:578040960.0 acc:-10.404301 acc1:-10.404301

[6, 6, 17, 17, 20, 32, 32, 13, 11, 11]
[63, 63, 64, 64, 64, 67, 65, 64, 64, 64]
###### 44 batch Train loss:1051616500.0 acc:-14.601664 acc1:-14.601664  Test loss:550592600.0 acc:-

[10, 8, 10, 8, 20, 35, 25, 9, 7, 16]
[39, 39, 40, 40, 41, 48, 43, 41, 41, 41]
###### 81 batch Train loss:514150100.0 acc:-8.314453 acc1:-8.314453  Test loss:197191710.0 acc:-4.3484874 acc1:-4.3484874

[16, 3, 15, 7, 13, 45, 23, 15, 16, 8]
[29, 29, 30, 30, 31, 38, 32, 31, 31, 31]
###### 82 batch Train loss:315902660.0 acc:-8.762471 acc1:-8.762471  Test loss:194493040.0 acc:-4.282756 acc1:-4.282756

[11, 6, 14, 18, 11, 31, 13, 6, 10, 9]
[35, 35, 37, 36, 37, 44, 39, 37, 37, 37]
###### 83 batch Train loss:273896860.0 acc:-3.8514588 acc1:-3.8514588  Test loss:191912660.0 acc:-4.2189493 acc1:-4.2189493

[10, 8, 21, 19, 9, 103, 25, 15, 16, 11]
[17, 17, 19, 18, 20, 27, 21, 19, 19, 19]
###### 84 batch Train loss:996790850.0 acc:-15.653779 acc1:-15.653779  Test loss:189434290.0 acc:-4.157344 acc1:-4.157344

[2, 3, 12, 9, 11, 23, 11, 6, 2, 5]
[11, 10, 12, 11, 13, 20, 14, 12, 12, 12]
###### 85 batch Train loss:495336220.0 acc:-9.893811 acc1:-9.893811  Test loss:187016640.0 acc:-4.0971775 acc1:-4.0

[3, 0, 6, 9, 8, 7, 6, 19, 14, 10]
[7, 6, 9, 8, 11, 22, 13, 10, 10, 10]
###### 122 batch Train loss:352206750.0 acc:-8.0525055 acc1:-8.0525055  Test loss:131117624.0 acc:-2.4721563 acc1:-2.4721563

[14, 11, 13, 11, 15, 46, 27, 7, 13, 18]
[32, 32, 35, 34, 36, 47, 38, 35, 35, 35]
###### 123 batch Train loss:558236160.0 acc:-7.7648454 acc1:-7.7648454  Test loss:130175320.0 acc:-2.4401338 acc1:-2.4401338

[0, 8, 12, 4, 7, 5, 9, 1, 2, 3]
[6, 6, 9, 8, 10, 21, 13, 9, 9, 9]
###### 124 batch Train loss:325414940.0 acc:-6.56923 acc1:-6.56923  Test loss:129246216.0 acc:-2.408509 acc1:-2.408509

[8, 1, 3, 10, 6, 10, 6, 24, 21, 8]
[27, 27, 29, 28, 31, 42, 33, 30, 30, 30]
###### 125 batch Train loss:254207630.0 acc:-7.5117006 acc1:-7.5117006  Test loss:128336370.0 acc:-2.3768132 acc1:-2.3768132

[42, 29, 108, 77, 133, 234, 205, 111, 96, 96]
[16, 16, 19, 18, 21, 32, 23, 19, 19, 20]
###### 126 batch Train loss:257695490.0 acc:-2.8993158 acc1:-2.8993158  Test loss:127446350.0 acc:-2.3459148 acc1:-2.3459

[13, 7, 13, 14, 24, 48, 39, 23, 11, 14]
[8, 8, 12, 10, 14, 29, 17, 12, 12, 13]
###### 163 batch Train loss:126723896.0 acc:-3.86448 acc1:-3.86448  Test loss:104161250.0 acc:-1.4401692 acc1:-1.4401692

[1, 6, 6, 1, 3, 5, 2, 0, 0, 7]
[26, 25, 29, 28, 32, 46, 35, 29, 30, 30]
###### 164 batch Train loss:170362100.0 acc:-2.53733 acc1:-3.5562408  Test loss:103765050.0 acc:-1.4210638 acc1:-1.4210638

[2, 3, 12, 9, 11, 23, 11, 6, 2, 5]
[24, 24, 28, 26, 30, 44, 33, 28, 28, 28]
###### 165 batch Train loss:179327310.0 acc:-2.8185756 acc1:-2.8185756  Test loss:103317064.0 acc:-1.4022527 acc1:-1.4022527

[12, 5, 12, 13, 21, 53, 25, 25, 15, 9]
[11, 11, 15, 13, 17, 32, 21, 15, 15, 16]
###### 166 batch Train loss:159457680.0 acc:-4.966455 acc1:-4.966455  Test loss:102893540.0 acc:-1.3839451 acc1:-1.3839451

[17, 4, 14, 14, 23, 55, 20, 19, 19, 8]
[1, 1, 3, 2, 5, 19, 8, 3, 3, 3]
###### 167 batch Train loss:142312200.0 acc:-2.461573 acc1:-3.498501  Test loss:102485130.0 acc:-1.3658515 acc1:-1.3658515

[1

In [82]:
for n in tf.get_default_graph().as_graph_def().node:
    try:
        print(tf.get_default_graph().get_tensor_by_name(n.name+':0'))
    except:
        print(n.name, "has 0 outputs")
        continue

Tensor("Placeholder:0", shape=(?, 19, 370, 370, 1), dtype=float32, device=/device:CPU:0)
Tensor("Placeholder_1:0", shape=(40, 370, 370), dtype=float32, device=/device:CPU:0)
Tensor("Placeholder_2:0", shape=(?, 370, 370, 1), dtype=float32, device=/device:CPU:0)
Tensor("Placeholder_3:0", shape=(?, 19, 1), dtype=float32, device=/device:CPU:0)
Tensor("Placeholder_4:0", shape=(?, 1), dtype=float32, device=/device:CPU:0)
Tensor("strided_slice/stack:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Tensor("strided_slice/stack_1:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Tensor("strided_slice/stack_2:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Tensor("strided_slice:0", shape=(?, 19, 370, 370, 1), dtype=float32, device=/device:GPU:0)
Tensor("strided_slice_1/stack:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Tensor("strided_slice_1/stack_1:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Tensor("strided_slice_1/stack_2:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Te

Tensor("gradients/CSTGCN/gcn_block_O_vision/einsum/Einsum_grad/add:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Tensor("gradients/CSTGCN/gcn_block_O_vision/einsum/Einsum_grad/mod:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Tensor("gradients/CSTGCN/gcn_block_O_vision/einsum/Einsum_grad/Shape_2:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Tensor("gradients/CSTGCN/gcn_block_O_vision/einsum/Einsum_grad/range/start:0", shape=(), dtype=int32, device=/device:GPU:0)
Tensor("gradients/CSTGCN/gcn_block_O_vision/einsum/Einsum_grad/range/delta:0", shape=(), dtype=int32, device=/device:GPU:0)
Tensor("gradients/CSTGCN/gcn_block_O_vision/einsum/Einsum_grad/range:0", shape=(3,), dtype=int32, device=/device:GPU:0)
Tensor("gradients/CSTGCN/gcn_block_O_vision/einsum/Einsum_grad/Fill/value:0", shape=(), dtype=int32, device=/device:GPU:0)
Tensor("gradients/CSTGCN/gcn_block_O_vision/einsum/Einsum_grad/Fill:0", shape=(1,), dtype=int32, device=/device:GPU:0)
Tensor("gradients/CSTGCN/gcn_blo

Tensor("gradients_2/CSTGCN_4/gcn_block_O_vision/conv2d/Conv2D/Conv2D_grad/tuple/control_dependency_1:0", shape=(1, 370, 1, 1), dtype=float32, device=/device:GPU:2)
Tensor("gradients_2/CSTGCN_4/gcn_block_O_vision/conv2d_1/Conv2D/Conv2D_grad/ShapeN:0", shape=(4,), dtype=int32, device=/device:GPU:2)
Tensor("gradients_2/CSTGCN_4/gcn_block_O_vision/conv2d_1/Conv2D/Conv2D_grad/Conv2DBackpropInput:0", shape=(?, 370, 370, 1), dtype=float32, device=/device:GPU:2)
Tensor("gradients_2/CSTGCN_4/gcn_block_O_vision/conv2d_1/Conv2D/Conv2D_grad/Conv2DBackpropFilter:0", shape=(370, 1, 1, 1), dtype=float32, device=/device:GPU:2)
gradients_2/CSTGCN_4/gcn_block_O_vision/conv2d_1/Conv2D/Conv2D_grad/tuple/group_deps has 0 outputs
Tensor("gradients_2/CSTGCN_4/gcn_block_O_vision/conv2d_1/Conv2D/Conv2D_grad/tuple/control_dependency:0", shape=(?, 370, 370, 1), dtype=float32, device=/device:GPU:2)
Tensor("gradients_2/CSTGCN_4/gcn_block_O_vision/conv2d_1/Conv2D/Conv2D_grad/tuple/control_dependency_1:0", shape=(37

Tensor("concat_1/axis:0", shape=(), dtype=int32, device=/device:CPU:0)
Tensor("concat_1:0", shape=(4, 19, 370, 370, 1), dtype=float32, device=/device:CPU:0)
Tensor("Mean_3/reduction_indices:0", shape=(), dtype=int32, device=/device:CPU:0)
Tensor("Mean_3:0", shape=(19, 370, 370, 1), dtype=float32, device=/device:CPU:0)
Tensor("ExpandDims_8/dim:0", shape=(), dtype=int32, device=/device:CPU:0)
Tensor("ExpandDims_8:0", shape=(1, 1, 370, 1, 1), dtype=float32, device=/device:CPU:0)
Tensor("ExpandDims_9/dim:0", shape=(), dtype=int32, device=/device:CPU:0)
Tensor("ExpandDims_9:0", shape=(1, 1, 370, 1, 1), dtype=float32, device=/device:CPU:0)
Tensor("ExpandDims_10/dim:0", shape=(), dtype=int32, device=/device:CPU:0)
Tensor("ExpandDims_10:0", shape=(1, 1, 370, 1, 1), dtype=float32, device=/device:CPU:0)
Tensor("ExpandDims_11/dim:0", shape=(), dtype=int32, device=/device:CPU:0)
Tensor("ExpandDims_11:0", shape=(1, 1, 370, 1, 1), dtype=float32, device=/device:CPU:0)
Tensor("concat_2/axis:0", shape=

In [81]:
tf.global_variables()

[<tf.Variable 'CSTGCN/gcn_block_O_vision/Im:0' shape=(40, 370, 370) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_O_vision/b:0' shape=(19, 370, 370, 1) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_O_vision/conv2d/kernel:0' shape=(1, 370, 1, 1) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_O_vision/conv2d/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_O_vision/wo:0' shape=(19, 370, 370, 1) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_O_vision/bo:0' shape=(19, 370, 370, 1) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_O_vision/conv2d_1/kernel:0' shape=(370, 1, 1, 1) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_O_vision/conv2d_1/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_O_vision/wd:0' shape=(19, 370, 370, 1) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_O_vision/bd:0' shape=(19, 370, 370, 1) dtype=float32_ref>,
 <tf.Variable 'CSTGCN/gcn_block_D_vision/Im:0' shape=(40, 370, 370) dtype=float32_